# 0. Constants & Helpers

In [5]:
import pandas as pd
import altair as alt
from copy import deepcopy

scale = 0.9

shape_circle = "M-10 0A1 1 0 0010 0 1 1 0 00-10 0"
shape_shoulders = "M-8-6C-15-2-15 2-8 6M8 6C15 2 15-2 8-6"
shape_face = "M0-10 0-10-4-9C0-13 0-13 4-9L0-10"

chart_dict = {
  "config": {"style": {"cell": {"stroke": "transparent"}}},
  "layer": [
    {
      "layer": [
        {
          "data": {"values": [{"x": 0}, {"x": 100}]},
          "mark": {
            "strokeDash": [scale*55, scale*55],
            "type": "rule",
            "strokeWidth": scale*6,
            "color": "whitesmoke",
          },
          "encoding": {
            "x": {"field": "x", "type": "quantitative", "axis": None}
          }
        },
        {
          "data": {"values": [{"xt": -50, "y": 0}]},
          "mark": {"type": "rule", "strokeWidth": scale*6, "color": "whitesmoke"},
          "encoding": {
            "x": {"field": "xt", "type": "quantitative", "axis": None}
          }
        }
      ]
    },
    {
      "layer": [
        {
          "mark": {
            "type": "point",
            "shape": shape_circle + shape_shoulders + shape_face,
            "filled": True,
            "opacity": 0.4,
            "size": scale*20
          }
        },
        {
          "mark": {
            "type": "point",
            "filled": True,
            "shape": shape_circle,
            "size": scale*20,
            "strokeWidth": scale*3,
            "opacity": 1
          }
        }
      ],
      "encoding": {
        "x": {
          "axis": None,
          "field": "x",
          "scale": {"domain": [0, 100]},
          "type": "quantitative"
        },
        "y": {
          "axis": None,
          "field": "y",
          "scale": {"domain": [20, 80]},
          "type": "quantitative"
        },
        "color": {
              "field": "Team",
              "scale": {"domain": ["D", "A"], "range": ["#d22", "#22d"]},
              "legend": None
        },
        "angle": {
              "field": "A",
              "type": "quantitative",
              "scale": {"domain": [-180, 180], "range": [180, 540]}
        }
      }
    },
    # {
    #   "layer": [
    #     {
    #         "mark": {
    #         "type": "point",
    #         "filled": False,
    #         "shape": shape_circle,
    #         "size": scale*20,
    #         "strokeWidth": scale*1,
    #         "opacity": 1,
    #         "color": "black"
    #       },
    #     }
    #   ],
    #   "encoding": {
    #     "x": {
    #       "axis": None,
    #       "field": "x",
    #       "scale": {"domain": [0, 100]},
    #       "type": "quantitative"
    #     },
    #     "y": {
    #       "axis": None,
    #       "field": "y",
    #       "scale": {"domain": [20, 80]},
    #       "type": "quantitative"
    #     },
    #     "angle": {
    #       "field": "A",
    #       "type": "quantitative",
    #       "scale": {"domain": [-180, 180], "range": [180, 540]}
    #     }
    #   }
    # },
    {
      "mark": {
        "type": "text",
        "align": "center",
        "color": "whitesmoke",
        "dy": 1,
        "fontSize": scale*26,
        "fontWeight": "bolder"
      },
      "encoding": {
        "text": {"field": "Label", "type": "nominal"},
        "x": {"axis": None, "field": "x", "type": "quantitative"},
        "y": {"axis": None, "field": "y", "type": "quantitative"}
      }
    },
    {
      "data": {"values": [{"x": 33, "y":34}, {"x": 48, "y": 34}, {"x":63, "y": 34}]},
      "mark": {
          "type": "point", 
          "shape": "arrow", 
          "size":(scale**2)*1500, 
          "filled":True, 
          "yOffset":-50*scale, 
          "fill":"black", 
          "opacity":1.0
      },
      "encoding": {
        "x": {"field": "x", "type": "quantitative"},
        "y": {"field": "y", "type": "quantitative"}
      }
    },
    {
      "data": {"values": [{"x": -50, "y": 50, "img": "https://raw.githubusercontent.com/samnlindsay/RugbyResults/master/ball_icon.png"}]},
      "mark": {"type": "image", "width": scale*25, "height": scale*25},
      "encoding": {
        "x": {"field": "x", "type": "quantitative"},
        "y": {"field": "y", "type": "quantitative"},
        "url": {"field": "img", "type": "nominal"}
      }
}
  ],
  "background": "#6c6",
  "data": {
    "values": None
  },
  "height": scale*400,
  "width": scale*500,
  "$schema": "https://vega.github.io/schema/vega-lite/v5.17.0.json"
}


def chart_from_data(df, arrows=None, chart_dict=chart_dict, ball_pos=[-50,50], as_schema=False):
    """
    Generate an Altair chart from the given data.

    Parameters:
    - df: pandas DataFrame
        The data to be visualized in the chart.
    - arrows: pandas DataFrame, optional
        The data for arrows to be displayed on the chart.
    - chart_dict: dict, optional
        The dictionary containing the chart configuration.
    - ball_pos: list, optional
        The position of the ball on the chart.
    - as_schema: bool, optional
        If True, returns the chart configuration as a dictionary (schema).
        If False, returns the chart object.

    Returns:
    - alt.Chart or dict
        The generated Altair chart or chart configuration.

    """
    schema = deepcopy(chart_dict)
    schema["data"]["values"] = df.to_dict(orient="records")
    schema["layer"][-1]["data"]["values"][0]["x"] = ball_pos[0]
    schema["layer"][-1]["data"]["values"][0]["y"] = ball_pos[1]
    if arrows is None:
      del schema["layer"][-2]
    else:
      schema["layer"][-2]["data"]["values"] = arrows.to_dict(orient="records")

    if as_schema:
      return schema
    else:
      return alt.Chart.from_dict(schema)



d_line_y = 56.0
a_line_y = 44.0
a_walkin_y = 34.0

min_x = 3
max_x = 95
mid_x = (min_x + max_x) / 2.0

a2 = (-52,50)
d2 = (-25,75)

a9 = (50,25)
d9 = (80,75)

setup_lookup = {
   "A": "Auckland",
   "C": "Crusaders",
   "H": "Highlanders",
   "W": "Waikato"
}

maul_dx = [0,-7, 7,-3, 6,  4, -4, 2]
maul_dy = [0, 0, 1,-5,-4,-9,-10,-14]

def spacing(type, n=7):
  """
  Calculate the spacing between lineout players.

  Parameters:
  - type (str): The type of spacing to calculate. Valid options are "even", "lift", "prelift", and "tight".
  - n (int): The number of players in the lineout. Default is 7.

  Returns:
  - float: The calculated spacing value.

  Raises:
  - ValueError: If an invalid spacing type is provided.

  """
  if type == "even":
    return (max_x - min_x) / (n-1)
  elif type == "lift":
    return 6.0
  elif type == "prelift":
    return 10.0
  elif type == "tight":
    return 14.0
  else:
    raise ValueError("Invalid spacing type provided.")
    
def defence(n=7, maul=False, x=None):
  """
  Creates a DataFrame representing the defensive line in a rugby lineout.

  Parameters:
  - n (int): The number of players in the defensive line (default: 7)

  Returns:
  - pd.DataFrame: A DataFrame containing the details of each player in the defensive line.
  """

  d_line = []
  d_line.append({"Team": "D", "x": max([-25, x-30]) if maul else d2[0], "y": d_line_y if maul else d2[1], "Label": "", "Jumper": "", "Order": -1, "A": 180})
  d_line.append({"Team": "D", "x": (x+30) if maul else d9[0], "y": d_line_y if maul else d9[1], "Label": "", "Jumper": "", "Order": 9, "A": 180})

  for i in range(1,n+1):
    if maul:
      d_line.append({"Team": "D", "x": x - maul_dx[i-1], "y": 53-maul_dy[i-1], "Label": "", "Jumper": "", "Order": i, "A": 180})
    else:
      x = min_x + spacing("even",n) * (i-1)
      angle = 90 if i==1 else (-90 if i==n else 180)
      d_line.append({"Team": "D", "x": x, "y": d_line_y, "Label": "", "Jumper": "", "Order": i, "A": angle})


  return pd.DataFrame(d_line)


# 1. Setup

In [6]:
def attack_init(n=7):
    """
    Initialize the attack line for a rugby match.

    Parameters:
    - n (int): Number of players in the attack line (default: 7)

    Returns:
    - pd.DataFrame: DataFrame representing the attack line with player information
    """

    a_line = [
        {"Team": "A", "x": a2[0], "y": a2[1], "Label": "2", "Jumper": "", "Order": -1, "A": 90},
        {"Team": "A", "x": a9[0], "y": a9[1], "Label": "9", "Jumper": "", "Order": 9, "A": 0},
    ]
    for i in range(1,n+1):
        x = min_x + min([spacing("even",n),20]) * (i-1)
        a_line.append({"Team": "A", "x": x, "y": a_line_y, "Label": "L" if i in [1,n] else "", "Jumper": "", "Order": i, "A": 90 if i==1 else -90})
        
    return pd.DataFrame(a_line)
    

def get_walk_ins(n, setup):
    """
    Returns a list of players who will participate in the lineout walk-in based on the given setup.

    Parameters:
    - n (int): The total number of players in the lineout.
    - setup (str): The setup type, which can be one of the following: "H" (Hooker), "C" (Center), "W" (Wing), or "A" (Alternate).

    Returns:
    - list: A list of player numbers who will participate in the lineout walk-in.

    Note:
    - For setup "H", all players except the first and last will participate in the walk-in.
    - For setup "C", the middle 3 players will participate in the walk-in.
    - For setup "W", the second and second-to-last players will always participate in the walk-in.
    - For setup "A", there is no walk-in.
    """
    if setup == "H":
        return list(range(2,n))
    elif setup == "C":
        if n > 5:
            return [n-4,n-3,n-2]
        else:
            return [2,3,4]
    elif setup == "W":
        if n > 5:
            return [2,n-3,n-1]
        else:
            return [2,4]
    elif setup == "A":
        return None
    
def get_arrows(df, walk_in):
    """
    Returns a DataFrame containing arrow coordinates based on the given DataFrame and walk-in values.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    walk_in (list): A list of walk-in values.

    Returns:
    DataFrame: A DataFrame containing arrow coordinates.

    Example:
    >>> df = pd.DataFrame({"Order": [1, 2, 3], "x": [10, 20, 30]})
    >>> walk_in = [2, 3]
    >>> get_arrows(df, walk_in)
       x
    0  20
    1  30
    """
    if walk_in is None:
        return None
    
    a = []
    for i in walk_in:
        x = df.loc[df["Order"]==i, "x"].values[0]
        a.append({"x": x, "y": a_walkin_y, })
    return pd.DataFrame(a)

# Order index of jumpers (these will be labelled with "J")
def jumpers(n, setup):
    """
    Determines the jumpers for a lineout based on the number of players and the setup.

    Parameters:
    - n (int): The number of players in the lineout.
    - setup (str): The setup of the lineout. [H, C, W, A]

    Returns:
    - list: A list of jumper positions in the lineout.

    """
    if setup == "H":
        if n > 5:
            return [2,n-3,n-1]
        else:
            return [2,3,4]
    elif setup == "C":
        if n == 7:
            return [2,4,n-1]
        elif n == 6:
            return [2, None, n-1]
        return [2,3,4]
    elif setup == "W":
        if n > 5:
            return [2,4,n-1]
        else:
            return [2,3,4]
    elif setup == "A":
        if n > 5:
            return [3,n-2,n-1]
        else:
            return [3,None,4]
    
def label_jumpers(df, jumpers):
    """
    Labels the jumpers in the given DataFrame based on the provided jumpers list.

    Args:
        df (DataFrame): The DataFrame containing the lineout data.
        jumpers (list): A list of jumpers in the order they jumped.

    Returns:
        DataFrame: The updated DataFrame with jumpers labeled and additional columns added.

    """
    for i,o in enumerate(jumpers):
        df.loc[df["Order"]==o, "Label"] = "J"
        
        df.loc[df["Order"]==o, "Jumper"] = ["2", "3", "1"][i]

    return df

def move_walkins(df, walk_in=None, a_walkin_y=a_walkin_y, n=7, setup="C"):
    """
    Move the walk-in players in the given DataFrame to specified coordinates.

    Parameters:
    - df: DataFrame - The input DataFrame containing the player data.
    - walk_in: list or None - The list of player orders to be moved. If None, no players will be moved.
    - a_walkin_y: float - The y-coordinate to which the walk-in players will be moved.
    - n: int - The total number of players in the team.
    - setup: str - The setup type of the team. [H, C, W, A]

    Returns:
    - DataFrame - The modified DataFrame with the walk-in players moved to the specified coordinates.
    """

    df["x"] = df["x"].astype(float)
    
    if walk_in is None:
        return df
    else: 
        move = df["Order"].isin(walk_in) & df["Team"].eq("A")
        
        # Move y-coordinate of walk-ins
        df.loc[move, "y"] = a_walkin_y
        df.loc[move, "A"] = 0
        
        # Move x-coordinate of walk-ins (if 3 in a row)
        if walk_in == [min(walk_in) + a for a in [0,1,2]] and (setup != "H"):
            dx = spacing("prelift")
            x = float(df.loc[df["Order"].isin([1,n]), "x"].values.mean())
            df.loc[move, "x"] = [x-dx, x, x+dx]
            
        return df
    

def adjust_spacing(df, n, setup):
    """
    Adjusts the spacing of data in a DataFrame based on the given parameters.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the data to be adjusted.
    n (int): The number of elements in the data.
    setup (str): The setup type. [H, C, W, A]
    Returns:
    pandas.DataFrame: The DataFrame with adjusted spacing.

    """
    ddx = spacing("prelift")
    dx = spacing("tight")
    
    if setup == "C": # 2-3-2 or 1-3-2 or 1-3-1
        if n > 5:
            df.loc[df["Order"]==n-1, "x"] = max_x - ddx
            if n==6:
                df.loc[df["Order"].isin([2,3,4]), "x"] -= ddx/2
            elif n==7:
                df.loc[df["Order"]==2, "x"] = min_x + ddx
    elif setup == "W":  
        if n > 5:
            df.loc[df["Order"]==4, "x"] = mid_x
        if n == 6:
            df.loc[df["Order"].isin([2,3,5]), "x"] = [mid_x-(2*dx), mid_x-dx, mid_x+dx]
    elif setup == "A": # 1-3-2-1 or 1-3-2 or 1-3-1
        dx = spacing("tight")
        ddx = spacing("prelift")
        df.loc[df["Order"].isin([2,3,4]), "x"] = [min_x + 2*dx + p for p in [-ddx, 0, ddx]]
        if n == 7:
            df.loc[df["Order"].isin([5,6]), "x"] = [min_x + 4.5*dx + 0.5*p for p in [-ddx, ddx]]
        elif n == 6:
            df.loc[df["Order"].isin([5,6]), "x"] = [min_x + 5*dx + p for p in [0, ddx]]
            # df.loc[df["Order"]==n, "x"] = max_x - dx
            # df.loc[df["Order"]==n-1, "x"] = mid_x
    
    return df

def adjust_angles(df, n, setup):
    if setup == "W":
        df.loc[df["Order"].isin(range(2,n)), "A"] = 0
    elif setup == "A":
        df.loc[df["Order"].isin([2,4]), "A"] = [60,-60]
    return df

def lineup(n=7, setup="H", receiver="9"):
    """
    Function to simulate a rugby lineup formation.

    Parameters:
    - n (int): Number of players in the lineup (default: 7)
    - setup (str): Setup of the lineup (default: "H")
    - receiver (str): Receiver position in the lineup (default: "9")

    Returns:
    - a (DataFrame): DataFrame representing the lineup formation
    - arrows (list): List of arrows representing the movement of players in the lineup
    """
    a = attack_init(n)
    walk = get_walk_ins(n, setup)
    jump = jumpers(n, setup)
    a = move_walkins(a, walk, n=n, setup=setup)
    a = label_jumpers(a, jump)
    a = adjust_spacing(a, n, setup)
    a = adjust_angles(a, n, setup)

    if setup == "A":
        a.loc[a["Order"]==2, "Label"] = "D"
    
    if receiver == "F":
        a.loc[a["Order"]==9, "Label"] = "+1"
    elif receiver is None:
        a = a[a["Order"] != 9]
    
    arrows = get_arrows(a, walk)
    return a, arrows

def setup_chart(n, setup, receiver="9"):
    """
    Creates a rugby chart based on the given lineup setup and receiver.

    Parameters:
    - n (int): The number of players in the lineup.
    - setup (str): The lineup setup.
    - receiver (str): The receiver position (default is "9").

    Returns:
    - chart: The rugby chart object.

    """
    df, arrows = lineup(n, setup, receiver)
    df = pd.concat([df, defence(n)])
    
    chart = chart_from_data(df, arrows)

    return chart

In [7]:
setup_chart(5, "W", "9")

alt.LayerChart(...)

In [262]:
n = 7
setup = "H"
call = 1

df = lineup(n, setup)[0]

def move_receiver1(df, call, receiver="9", play=None):
    
    if receiver:
        x_j = df.loc[df["Jumper"]==str(call), "x"].values[0]
        df.loc[df["Order"]==9, "y"] = 30
    
        if play is None:
            if receiver == "9":
                play = "Hot"
            else:
                play = "Cold"
                
        if receiver == "9" and play == "Hot":
            df.loc[df["Order"]==9, "x"] = (x_j + a9[0])/2
    
        if receiver == "F" and play in ["Cold", "Flyby", "Crusaders"]:
            df.loc[df["Order"]==9, "x"] = x_j
            df.loc[df["Order"]==9, "y"] = a_line_y - 12
    
    return df

def setup_to_throw(df, n=7, setup="H", call=1):
    """
    Adjusts the positions of players in a rugby lineout setup based on the specified parameters.

    Parameters:
    - df: DataFrame - The input DataFrame containing player positions.
    - n: int - The total number of players in the lineout (default: 7).
    - setup: str - The setup type, options are "H" (Highlanders), "C" (Canterbury), "A" (Auckland), or "W" (Waikato) (default: "H").
    - call: int - The call number, options are 1, 2, or 3 (default: 1).

    Returns:
    - DataFrame: The updated DataFrame with adjusted player positions.
    """
    df.loc[df["y"]==a_walkin_y, "y"] = a_line_y 
    dx = spacing("lift")
    ddx = spacing("prelift")
    
    ###############
    # HIGHLANDERS #
    ###############
    if setup == "H":
        # Move players either side of Jumper {call}
        if call == 2:
            j = df.loc[df["Jumper"]==str(call), "Order"].values[0]
            x = df.loc[df["Order"]==(j-1), "x"].values[0]
            pod = [j-1, j, j+1]
            df.loc[df["Order"].isin(pod), "x"] = [x, x+dx, x+(2*dx)]
        else:
            j = df.loc[df["Jumper"]==str(call), "Order"].values[0]
            x = df.loc[df["Order"]==j, "x"].values[0]
            pod = [j-1, j, j+1]
            df.loc[df["Order"].isin(pod), "x"] = [x-dx, x, x+dx]
        
        df.loc[df["Order"].isin(pod), "A"] = [90,-90,-90]
    
    ##############
    # CANTERBURY #
    ##############
    elif setup == "C":
        # Back pod
        j1 = df.loc[df["Jumper"]=="1", "Order"].values[0]
        if n > 5:
            x1 = df.loc[df["Order"]==j1, "x"].values[0]
        else:
            x1 = max_x - spacing("even", n)
        pod1 = [j1-1, j1, j1+1]
        df.loc[df["Order"].isin(pod1), "A"] = [90,-90,-90]

        # Front pod
        j2 = df.loc[df["Jumper"]=="2", "Order"].values[0]
        x2 = df.loc[df["Order"]==(j2-1), "x"].values[0]
        pod2 = [j2-1, j2, j2+1]
        df.loc[df["Order"].isin(pod2), "A"] = [90,-90,-90]

        dx1 = dx if call == 1 else ddx
        dx2 = dx if call == 2 else ddx

        if n > 5:
            df.loc[df["Order"].isin(pod1), "x"] = [x1-dx1, x1, x1+dx1]
            df.loc[df["Order"].isin(pod2), "x"] = [x2, x2+dx2, x2+(2*dx2)]
        else:
            if call == 1:   
                df.loc[df["Order"].isin(pod1), "x"] = [x1-dx1, x1, x1+dx1]
                df.loc[df["Order"].isin(pod2[:2]), "x"] = [x2, x2+dx2]
            elif call in [2,3]:
                df.loc[df["Order"].isin(pod1[1:]), "x"] = [x1, x1+dx1]
                df.loc[df["Order"].isin(pod2), "x"] = [x2, x2+dx2, x2+(2*dx2)]
        j = j1 if call == 1 else j2

        # Middle pod
        if call == 3:
            j = df.loc[df["Jumper"]==("3" if n!=6 else "1"), "Order"].values[0]
            if n < 7:
                pod = [1, j, j+1]
            else:
                pod = [1, j, n-1]
            x = mid_x if n > 5 else (min_x + df.loc[df["Order"]==4, "x"].values[0])/2
            df.loc[df["Order"].isin(pod), "x"] = [mid_x-dx, mid_x, mid_x+dx]
            df.loc[df["Order"].isin(range(2,n)), "A"] = -30
            df.loc[df["Order"].isin(pod), "A"] = [90,-90,-90]
            
    ############
    # AUCKLAND #
    ############
    elif setup == "A":        
        # Front jumper
        df.loc[df["Jumper"]=="2", "x"] = min_x + dx if call == 2 else min_x + ddx
        
        # Dummy jumper always moves out and back behind the front pod
        df.loc[df["Order"]==2, "x"] = (min_x + 2*dx + spacing("tight")) if call == 2 else mid_x

        if n == 7:
            # Middle man always supports J2
            df.loc[df["Order"]==4, "x"] = min_x + 2*ddx
    
        if call==2:
            j = df.loc[df["Jumper"]==str(call), "Order"].values[0]
            pod = [1, j, j+1]
            df.loc[df["Order"].isin(pod), "x"] = [min_x, min_x+dx, min_x+2*dx]
            df.loc[df["Order"].isin(pod), "A"] = [90,-90,-90]
        elif call==3:
            j = df.loc[df["Jumper"]==("3" if n>5 else "1"), "Order"].values[0]
            pod = [2, j, j+1]
            df.loc[df["Order"].isin(pod), "x"] = [mid_x-dx, mid_x, mid_x+dx]
            df.loc[df["Order"].isin(pod), "A"] = [90,-90,-90]
        elif call==1:
            j = df.loc[df["Jumper"]==str(call), "Order"].values[0]
            if n>5:
                x = df.loc[df["Order"]==j, "x"].values[0]
                pod = [j-1, j, j+1]
            else:
                x = df.loc[df["Order"]==n, "x"].values[0] - spacing("even", n)
                pod = [2, j, n]
            df.loc[df["Order"].isin(pod), "x"] = [x-dx, x, x+dx]
            df.loc[df["Order"].isin(pod), "A"] = [90,-90,-90]

        df.loc[df["Order"]==2, "A"] = -30 if call == 2 else 90

    ###########
    # WAIKATO #
    ###########
    elif setup == "W":
        j = df.loc[df["Jumper"]==str(call), "Order"].values[0]
        pod = [j-1, j, j+1]
        if call == 2:
            x = df.loc[df["Order"]==j-1, "x"].values[0]
            df.loc[df["Order"].isin(pod), "x"] = [x, x+dx, x+2*dx]
        else:
            x = df.loc[df["Order"]==j, "x"].values[0]
            if n==6 and call==1:
                x = max_x - spacing("even", n)
            df.loc[df["Order"].isin(pod), "x"] = [x-dx, x, x+dx]
        df.loc[df["Order"].isin(pod), "A"] = [90,-90,-90]

    # Move hooker one third of the horizontal distance to the jumper
    x_j = df.loc[df["Order"]==j, "x"].values[0]
    x_h = df.loc[df["Order"]==-1, "x"].values[0]

    df.loc[df["Order"]==-1, "x"] = (x_j - x_h)/3 + x_h
    df.loc[df["Order"]==-1, "y"] = a_line_y - 10
    df.loc[df["Order"]==-1, "A"] += 10

    return df

def throw_lineup(n, setup, call, receiver="9", play=None):
    df, arrows = lineup(n, setup, receiver)
    df.loc[df["Order"].isin(range(2,n+1)), "A"] = -90
    df = setup_to_throw(df, n, setup, call)

    if n==6 and setup=="C" and call==3:
        # call = "1"
        df.loc[df["Jumper"]=="1", "Jumper"] = "3"
    if n==5 and setup=="A" and call==3:
        # call = "1"
        df.loc[df["Jumper"]=="1", "Jumper"] = "3"

    df = move_receiver1(df, call, receiver, play)
    return df

def throw_chart(n, setup, call, receiver="9", play=None):
    """
    Generates a chart representing the throw in a rugby lineout.

    Parameters:
    - n (int): The number of players in the lineout.
    - setup (str): The setup position of the lineout.
    - call (int or str): The call made by the hooker.
    - receiver (str): The position of the receiver. Default is "9".

    Returns:
    - chart (Chart): The chart representing the lineout throw.

    """
    df = throw_lineup(n, setup, call, receiver, play)
    ball_pos = [
        df.loc[df["Jumper"]==str(call), "x"].values[0] - 2,
        df.loc[df["Jumper"]==str(call), "y"].values[0] + 4,
    ]

    
    df = pd.concat([df, defence(n)])
    
    chart = chart_from_data(df, ball_pos=ball_pos)

    return chart

In [263]:
display(setup_chart(7, "A", "9"))
throw_chart(7, "A", 1, receiver="9")

alt.LayerChart(...)

alt.LayerChart(...)

# 3. Play

In [294]:
n=7
setup="A"
call=2
receiver="9"
play="Cold"

# Starting from the throw lineup
df = throw_lineup(n, setup, call, receiver)

# Remove labels from jumpers ("J") and lifters ("L")
df.loc[df["Label"].isin(["L", "D"]), "Label"] = ""
df.loc[~df["Jumper"].isin(["", str(call)]), "Label"] = ""

df.loc[df["Order"].isin([-1,9]), "Order"] = [-1,9]
df.loc[df["Order"].isin(range(1,n+1)), "Order"] = df.loc[df["Order"].isin(range(1,n+1)), "x"].rank(method="min", ascending=True)
df = df.sort_values("Order")

df

def move_receiver2(df, call, receiver="9", play=None):
    if play is None:
        if receiver == "9":
            play = "Hot"
        else:
            play = "Cold"
    
    if receiver == "F" and play != "Hot":
        df.loc[df["Order"]==9, "y"] = a_line_y - 4
        df.loc[df["Order"]==9, "x"] += 1 if call > 1 else -1    
    elif receiver == "9" and play in ["Crusaders", "Cold"]:
        x = df.loc[df["Jumper"]==str(call), "x"].values[0]
        df.loc[df["Order"]==9, "x"] = x + 12
    elif play == "Hot":
        x = df.loc[df["Jumper"]==str(call), "x"].values[0]
        df.loc[df["Order"]==9, "x"] = x
    
    return df

def maul_order(df, n, call, receiver, play="Cold"):
    
    j = df.loc[df["Jumper"]==str(call), "Order"].values[0]
    
    # Standard maul - use the +1
    if play in ["Cold", "Flyby"]:
        if receiver=="F":
            r = 9
        else:
            # 3rd closest player (absolute difference in x) to the jumper (not including the hooker)
            r = df.loc[~df["Order"].isin([j,-1,9]), "x"].sub(df.loc[df["Order"]==j, "x"].values[0]).abs().nsmallest(3).index[-1]
            print(r)

    # Otherwise choose the closest player to the pod
    elif play == "Crusaders":
        # ...as long as one side isn't overloaded
        open = len(df.loc[(df["Order"] > j+1) & (df["Label"] != 9)])
        blind = len(df.loc[df["Order"] < j-1])
        
        if abs(open - blind) >= 2:
            r = j+2 if open > blind else j-2  
            return r

    if call == 1:
        r = n-3
    elif call == 2:
        r = 4
    elif call == 3:
        r = j-2        

    return r

def maul(df, n, call, receiver, size, play="Cold"):

    x0 = df.loc[df["Jumper"]==str(call), "x"].values[0]
    y0 = df.loc[df["Jumper"]==str(call), "y"].values[0] + 3
    x_j = df.loc[df["Jumper"]==str(call), "x"].values[0]


    # Initiating the maul (pod + 1)
    j = df.loc[df["Jumper"]==str(call), "Order"].values[0]
    r = maul_order(df, n, call, receiver, play)
    pod = [j,j-1,j+1, r]

    # Sort df by distance from jumper
    df["dist"] = df.apply(lambda p: abs(p["x"]-x_j) if p["Order"] not in pod else -10, axis=1)
    df.loc[df["Order"].isin(pod), "dist"] += [pod.index(o) for o in df.loc[df["Order"].isin(pod), "Order"].values]
    
    # Scrum Half always bottom of list
    df.loc[df["Label"]=="9", "dist"] = 1000
    df = df.sort_values("dist").reset_index(drop=True)
    df = df.drop("dist", axis=1)
    
    # Replace first n x,y values with maul values
    # Move any additional players halfway to new position
    np = len(df) - 1 if receiver == "9" else len(df)

    for i in range(np):
        nx = x0 + maul_dx[i]
        ny = y0 + maul_dy[i]
        if i >= size:
            if play == "Crusaders":
                df.loc[i, "x"] = (df.loc[i, "x"] + nx) / 2
                df.loc[i, "y"] = (df.loc[i, "y"] + ny) / 2
                df.loc[i, "A"] = 0
            elif play == "Flyby":
                df.loc[i, "x"] = (df.loc[i, "x"] + 3*nx) / 4
                df.loc[i, "y"] = ny
                df.loc[i, "A"] = -30 if df.loc[i, "x"] > x_j else 30
                # # Leave pillars
                # else:
                #     df.loc[i, "x"] += 15 if i==pillars[0] else -15
                #     df.loc[i, "y"] -= 15 
                #     df.loc[i, "A"] = 30 if i==pillars[0] else -30
        else:
            df.loc[i, "x"] = nx
            if play == "Flyby" and i == 3:
                df.loc[i, "A"] = -135
                df.loc[i, "y"] = ny - 3
            else:
                df.loc[i, "A"] = 0
                df.loc[i, "y"] = ny
                
    return df

def crusaders_positions(df, n, call, receiver):
    # Mini maul
    df = maul(df, n, call, receiver, size=4, play="Crusaders")

    # Pod 
    x_j = df.loc[df["Jumper"]==str(call), "x"].values[0]
    j = df.loc[df["Jumper"]==str(call), "Order"].values[0]

    # Blind (hooker)
    df.loc[df["Order"]==-1, "x"] = max([-30, x_j-80])
    df.loc[df["Order"]==-1, "y"] = 30

    # Additional players
    r = maul_order(df, n, call, receiver, play="Crusaders")   
    # Open side
    open = sorted(df.loc[(df["Order"] > j+1) & (~df["Order"].isin([r])), "Order"].tolist())
    blind = sorted(df.loc[(df["Order"] < j-1) & (~df["Order"].isin([r])), "Order"].tolist())

    if len(open) - len(blind) > 1:
        blind.append(open.pop(0))

    if receiver == "9":
        df.loc[df["Order"]==9, "x"] = x_j + 10
        df.loc[df["Order"]==9, "y"] = 25

    if len(blind) > 1:
        df.loc[df["Order"]==blind[1], "x"] = max([-10, x_j-50])
        df.loc[df["Order"]==blind[1], "y"] = 25
        if len(blind) > 2:
            df.loc[df["Order"]==-1, "y"] = 25
            df.loc[df["Order"]==blind[1], "y"] += 5
            df.loc[df["Order"]==blind[2], "x"] = max([20, x_j-30])
            df.loc[df["Order"]==blind[2], "y"] = 25
            if len(blind) > 3:
                df.loc[df["Order"]==blind[3], "x"] = min([x_j+30, 120])
                df.loc[df["Order"]==blind[3], "y"] = 25

    if len(open) > 0:
        df.loc[df["Order"]==open[0], "x"] = min([x_j+50, 120])
        df.loc[df["Order"]==open[0], "y"] = 30
        if len(open) > 1:
            df.loc[df["Order"]==open[1], "x"] = min([x_j+30, 100])
            df.loc[df["Order"]==open[1], "y"] = 25
            if len(open) > 2:
                df.loc[df["Order"]==open[2], "x"] = min([x_j+70, 140])
                df.loc[df["Order"]==open[2], "y"] = 25

    return df

def flyby_positions(df, n, call, receiver):
    # Mini maul
    df = maul(df, n, call, receiver, size=4, play="Flyby")

    # Pod 
    x_j = df.loc[df["Jumper"]==str(call), "x"].values[0]

    # Hooker
    df.loc[df["Order"]==-1, "x"] = x_j - 10
    df.loc[df["Order"]==-1, "y"] = a_line_y - 15
    df.loc[df["Order"]==-1, "A"] = 80

    if receiver == "9":
        df.loc[df["Order"]==9, "x"] += 20 if call == 2 else 40
        df.loc[df["Order"]==9, "y"] -= 10

    return df

In [295]:
def play_chart(n, setup, call, receiver="9", play=None, arrows=None):
    """
    Generates a chart representing the play in a rugby lineout.

    Parameters:
    - n (int): The number of players in the lineout.
    - setup (str): The setup position of the lineout.
    - call (int or str): The call made by the hooker.
    - receiver (str): The position of the receiver. Default is "9".

    Returns:
    - chart (Chart): The chart representing the lineout play.

    """
    df = throw_lineup(n, setup, call, receiver, play)

    # Remove labels from jumpers ("J") and lifters ("L")
    # df.loc[df["Label"].isin(["L", "D"]), "Label"] = ""
    # df.loc[~df["Jumper"].isin(["", str(call)]), "Label"] = ""
    
    # df.loc[df["Order"].isin([-1,9]), "Order"] = [-1,9]
    df.loc[df["Order"].isin(range(1,n+1)), "Order"] = df.loc[df["Order"].isin(range(1,n+1)), "x"].rank(method="min", ascending=True)
    df = df.sort_values("Order")

    # Blind (hooker)
    df.loc[df["Order"]==-1, "x"] = -20
    df.loc[df["Order"]==-1, "y"] = 30

    df = move_receiver2(df, call, receiver, play)

    if play == "Cold":
        size = n + 1 if receiver != "F" else n + 2
        df = maul(df, n, call, receiver, size, play)

        # Position ball relative to the hindmost player
        min_y = min(df.loc[df["Order"]!=9, "y"])
        x = df.loc[df["y"]==min_y, "x"].values[0]
        ball_pos = [
            x + (-3 if size % 2 == 0 else 3),
            min_y #+ (0 if size % 2 == 0 else 2)
        ]
    elif play == "Crusaders":
        df = crusaders_positions(df, n, call, receiver)
        ball_pos = [
            df.loc[df["Jumper"]==str(call), "x"].values[0],
            df.loc[df["Jumper"]==str(call), "y"].values[0] - 3
        ]
    elif play == "Flyby":
        df = flyby_positions(df, n, call, receiver)
        ball_pos = [
            df.loc[df["Order"]==-1, "x"].values[0] + 3,
            df.loc[df["Order"]==-1, "y"].values[0] + 2
        ]
    elif play == "Hot":
        if receiver is None:
            r = maul_order(df, n, call, receiver)
            j_x = df.loc[df["Jumper"]==str(call), "x"].values[0]
            r_x = df.loc[df["Order"]==r, "x"].values[0]
            df.loc[df["Order"]==r, "x"] = (3*j_x + r_x)/4
            df.loc[df["Order"]==r, "y"] -= 15
            ball_pos = [
                df.loc[df["Order"]==r, "x"].values[0] - 2,
                df.loc[df["Order"]==r, "y"].values[0] + 4,
            ]
        else:
            df.loc[df["Order"]==9, "A"] = 30
            ball_pos = [
                df.loc[df["Order"]==9, "x"].values[0],
                df.loc[df["Order"]==9, "y"].values[0] + 4,
            ]
    if play == "Hot":
        df = pd.concat([df, defence(n)])
    else:
        df = pd.concat([defence(n, maul=True, x=df.loc[df["Jumper"]==str(call), "x"].values[0]), df])

    chart = chart_from_data(df, ball_pos=ball_pos, arrows=arrows)

    return chart

In [296]:
play_chart(7, "C", 1, receiver="9", play="Flyby")

5


,Team,x,y,Label,Jumper,Order,A
0,A,85.00,47.0,J,1,6,0
1,A,78.00,47.0,,,5,0
2,A,92.00,48.0,L,,7,0
3,A,82.00,39.0,J,3,4,-135
4,A,74.00,43.0,,,3,30
5,A,70.00,38.0,J,2,2,30
6,A,61.50,37.0,L,,1,30
7,A,60.25,33.0,2,,-1,30
8,A,50.00,30.0,9,,9,0


alt.LayerChart(...)

# 4. Save charts etc.

In [297]:
path = "json/angle/"

def save_charts(ns=[5,6,7], setups=["A", "C", "H", "W"], calls=[1,2,3], plays=["Hot", "Cold", "Crusaders", "Flyby"]):
    
    for n in ns: 
        for setup in setups:
            ################
            # SETUP CHARTS #
            ################
            print(n, setup)
            # Scrum Half
            chart = setup_chart(n, setup)
            chart.save(path + f"setup_{n}_{setup}.json", embed_options={"actions": False, "modebar": "none"})
            # No receiver
            chart = setup_chart(n, setup, receiver=None)
            chart.save(path + f"setup_{n}no9_{setup}.json", embed_options={"actions": False, "modebar": "none"})
    
            if n < 7:
                # Forward receiver
                chart = setup_chart(n, setup, receiver="F")
                chart.save(path + f"setup_{n}+1_{setup}.json", embed_options={"actions": False, "modebar": "none"})
    
            for call in calls:
                ################
                # THROW CHARTS #
                ################
                print(f"\t{call}")
    
                # Scrum Half
                chart = throw_chart(n, setup, call)
                chart.save(path + f"throw_{n}_{setup}{call}.json", embed_options={"actions": False, "modebar": "none"})
    
                # No receiver
                chart = throw_chart(n, setup, call, receiver=None)
                chart.save(path + f"throw_{n}no9_{setup}{call}.json", embed_options={"actions": False, "modebar": "none"})
    
                if n < 7:
                    # Forward receiver
                    chart = throw_chart(n, setup, call, receiver="F")
                    chart.save(path + f"throw_{n}+1_{setup}{call}.json", embed_options={"actions": False, "modebar": "none"})
    
                for play in plays:
                    ###############
                    # PLAY CHARTS #
                    ###############
                    print(f"\t\t{play}")
            
                    # Scrum Half
                    chart = play_chart(n, setup, call, play=play, receiver="9")
                    chart.save(path + f"play_{n}_{setup}{call}_{play}.json", embed_options={"actions": False, "modebar": "none"})
    
                    # No receiver
                    chart = play_chart(n, setup, call, play=play, receiver=None)
                    chart.save(path + f"play_{n}no9_{setup}{call}_{play}.json", embed_options={"actions": False, "modebar": "none"})
    
                    if n < 7:
                        # Forward receiver
                        chart = play_chart(n, setup, call, play=play, receiver="F")
                        chart.save(path + f"play_{n}+1_{setup}{call}_{play}.json", embed_options={"actions": False, "modebar": "none"})

In [298]:
save_charts(setups=["A"])

5 A
	1
		Hot
4
		Cold
4


,Team,x,y,Label,Jumper,Order,A
0,A,60.0,47.0,J,1,4,0
1,A,53.0,47.0,D,,3,0
2,A,67.0,48.0,L,,5,0
3,A,57.0,42.0,J,2,2,0
4,A,66.0,43.0,L,,1,0
5,A,64.0,38.0,2,,-1,0
6,A,72.0,30.0,9,,9,0


4


,Team,x,y,Label,Jumper,Order,A
0,A,60.0,47.0,J,1,4,0
1,A,53.0,47.0,D,,3,0
2,A,67.0,48.0,L,,5,0
3,A,57.0,42.0,J,2,2,0
4,A,66.0,43.0,L,,1,0
5,A,64.0,38.0,2,,-1,0


,Team,x,y,Label,Jumper,Order,A
0,A,60.0,47.0,J,1,4,0
1,A,53.0,47.0,D,,3,0
2,A,67.0,48.0,L,,5,0
3,A,57.0,42.0,J,2,2,0
4,A,66.0,43.0,+1,,9,0
5,A,64.0,38.0,L,,1,0
6,A,56.0,37.0,2,,-1,0


...

,Team,x,y,Label,Jumper,Order,A
0,A,49.0,47.0,J,3,4,0
1,A,42.0,47.0,D,,3,0
2,A,56.0,48.0,L,,5,0
3,A,46.0,42.0,J,2,2,0
4,A,55.0,43.0,L,,1,0
5,A,53.0,38.0,2,,-1,0


,Team,x,y,Label,Jumper,Order,A
0,A,49.0,47.0,J,3,4,0
1,A,42.0,47.0,D,,3,0
2,A,56.0,48.0,L,,5,0
3,A,46.0,42.0,J,2,2,0
4,A,55.0,43.0,+1,,9,0
5,A,53.0,38.0,L,,1,0
6,A,45.0,37.0,2,,-1,0


		Crusaders


,Team,x,y,Label,Jumper,Order,A
0,A,49.0,47.0,J,3,4,0
1,A,42.0,47.0,D,,3,0
2,A,56.0,48.0,L,,5,0
3,A,46.0,42.0,J,2,2,0
4,A,29.0,43.5,L,,1,0
5,A,16.5,34.0,2,,-1,0
6,A,61.0,30.0,9,,9,0


KeyboardInterrupt: 